# Modelos usando el algoritmo de Naive Bayes

## Importación de bibliotecas

In [1]:
import pandas as pd
import numpy as np
import preprocessing

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import RandomizedSearchCV

from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.metrics import accuracy_score, precision_score, roc_curve, f1_score, recall_score, plot_confusion_matrix, auc

import seaborn as sns
from matplotlib import pyplot as plt
sns.set()

In [2]:
GSPREADHSEET_DOWNLOAD_URL = (
    "https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format
)

FIUFIP_2021_1_GID = '1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0'
df = pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=FIUFIP_2021_1_GID))

## Llamado a funciones de preprocesamiento

In [3]:
df = preprocessing.remove_irrelevant_features(df)
df = preprocessing.missings_treatment(df)
df = preprocessing.one_hot_encodding(df)

# Se separa el dataset en entrenamiento y holdout
X_train, X_holdout, y_train, y_holdout = preprocessing.dataset_split(df)

## Funciones para gráficos

In [4]:
def plot_roc(modelo, X, y):

    fpr, tpr, thresholds = roc_curve(y, modelo.predict_proba(X)[:,1]) # aca le paso las probabilidades en vez de y_pred, es lo mas recomendado
    
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(15, 10))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [5]:
def plot_matriz_de_confusion(modelo, X, y):
    
    fig, ax = plt.subplots(figsize=(15, 7))
    plt.grid(False)
    plot_confusion_matrix(modelo, X, y, cmap=plt.cm.Blues, display_labels=['1', '0'], ax=ax)
    plt.title('Matriz de confusion')
    plt.show()

## Modelos

### Modelo 1
- se tratan todos los features como continuos

In [ ]:
modelo1 = GaussianNB()

smoothing = np.array([1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05,1e-04, 1e-03, 1e-02, 1e-01, 1])
params = {'var_smoothing': smoothing}


rscv = RandomizedSearchCV(
    modelo1, params, n_iter=11, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

print("Resultados modelo 1")
print(f"    Mejores hiperparámetros: {rscv.best_params_}")
print(f"    Métrica AUC ROC: {rscv.best_score_}")

modelo1 = rscv.best_estimator_

### Modelo 2
- se tratan todos los features como discretos 
- se descarta el feature ganancia_perdida_declarada_bolsa_argentina ya que toma demasiados valores y ademas tiene valores negativos que no son válidos para el algoritmo

In [ ]:
X_train_m = X_train.drop('ganancia_perdida_declarada_bolsa_argentina', axis='columns')

modelo2 = MultinomialNB()

smoothing = np.array([1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05,1e-04, 1e-03, 1e-02, 1e-01, 1])
params = {'alpha': smoothing}

rscv = RandomizedSearchCV(
    modelo2, params, n_iter=11, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train_m, y_train)

print("Resultados modelo 2")
print(f"    Mejores hiperparámetros: {rscv.best_params_}")
print(f"    Métrica AUC ROC: {rscv.best_score_}")

modelo2 = rscv.best_estimator_

### Modelo 3
- se usa una combinación de naive bayes multinomial y gaussiano
- primero se debe construir una clase estimador siguiendo las reglas de sklearn por cuestiones de compatibilidad

In [ ]:
class Modelo3(BaseEstimator, ClassifierMixin):
    def __init__(self, var_smoothing=1e-09, alpha=1): 
        self.var_smoothing = var_smoothing 
        self.alpha = alpha
        
    def fit(self, X_train, y_train):

        self._gnb = GaussianNB(var_smoothing = self.var_smoothing)
        self._mnb = MultinomialNB(alpha=self.alpha)
        
        X_train_g = X_train[['ganancia_perdida_declarada_bolsa_argentina']]
        X_train_m = X_train.drop('ganancia_perdida_declarada_bolsa_argentina', axis='columns')

        self._gnb.fit(X_train_g, y_train)
        self._mnb.fit(X_train_m, y_train)

        X_train_proba_g = pd.DataFrame(self._gnb.predict_proba(X_train_g)).drop(0, axis='columns')
        X_train_proba_m = pd.DataFrame(self._mnb.predict_proba(X_train_m)).drop(0, axis='columns')

        # se construye el dataframe de probabilidades con los datos de entrenamiento
        X_train_proba = pd.DataFrame()
        X_train_proba['1_gaussian'] = X_train_proba_g[1]
        X_train_proba['1_multinomial'] = X_train_proba_m[1]

        # se aplica naive bayes gaussiano al dataframe de probabilidades
        self._gnb_proba = GaussianNB().fit(X_train_proba, y_train)
        return self

    def predict(self, X_pred):
        X_pred_proba = self._construir_instancias_de_probabilidades(X_pred)
        return self._gnb_proba.predict(X_pred_proba)
    
    def predict_proba(self, X_pred):
        X_pred_proba = self._construir_instancias_de_probabilidades(X_pred)
        return self._gnb_proba.predict_proba(X_pred_proba)

    def _construir_instancias_de_probabilidades(self, X_pred):
        X_pred_g = X_pred[['ganancia_perdida_declarada_bolsa_argentina']]
        X_pred_m = X_pred.drop('ganancia_perdida_declarada_bolsa_argentina', axis='columns')

        # se aplica el modelo compuesto a los datos de validación y se obtiene la predicción
        X_pred_proba_g = pd.DataFrame(self._gnb.predict_proba(X_pred_g)).drop(0, axis='columns')
        X_pred_proba_m = pd.DataFrame(self._mnb.predict_proba(X_pred_m)).drop(0, axis='columns')
        X_pred_proba = pd.DataFrame()
        X_pred_proba['1_gaussian'] = X_pred_proba_g[1]
        X_pred_proba['1_multinomial'] = X_pred_proba_m[1]
        return X_pred_proba

In [ ]:
modelo3 = Modelo3()

smoothing = np.array([1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05,1e-04, 1e-03, 1e-02, 1e-01, 1])
params = {'var_smoothing': smoothing, 'alpha': smoothing}

rscv = RandomizedSearchCV(
    modelo3, params, n_iter=60, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

print("Resultados modelo 3")
print(f"    Mejores hiperparámetros: {rscv.best_params_}")
print(f"    Métrica AUC ROC: {rscv.best_score_}")

modelo3 = rscv.best_estimator_

## Conclusion
    En base a la metrica AUC-ROC se elige el modelo 1
### Informe modelo 1 usando los datos del test_holdout

In [ ]:
y_pred = modelo1.predict(X_holdout)
y_pred_proba = modelo1.predict_proba(X_holdout)

print('Metricas:')
print(f'    accuracy: {accuracy_score(y_holdout, y_pred)}')
print(f'    precision: {precision_score(y_holdout, y_pred)}')
print(f'    recall: {recall_score(y_holdout, y_pred, pos_label=0)}')
print(f'    f1 score: {f1_score(y_holdout, y_pred)}')

plot_matriz_de_confusion(modelo1, X_holdout, y_holdout)
plot_roc(modelo1, X_holdout, y_holdout)